In [1]:
from py2neo import Graph
import pandas as pd
import numpy as np
import os

In [2]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

In [3]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [8]:
def create_nodes(df, season):
    """Function to create the nodes from the csv files.

    Returns:
        str: Acknowledgement string
    """
    i=0
    for idx in df.index:
        id_ = df.loc[idx, 'Id']
        label = df.loc[idx, 'Label'].replace('\'', '')
        cypher = "MERGE (a:Person {id:" + "'" + \
            id_ + "'" + ",label:" + "'" + label + "'})"
        cypher = cypher + f'SET a.seed={i} '
        cypher = cypher + f'SET a:Person_{season}'
        graph.run(cypher)
        i+=1
    return "Done creating nodes"

In [9]:
def create_edges(df):
    """Function to create the edges from the csv files.

    Returns:
        str: Acknowledgement string
    """
    for idx in df.index:
        src = df.loc[idx, 'Source']
        tar = df.loc[idx, 'Target']
        weight = df.loc[idx, 'Weight']
        season = df.loc[idx, 'Season']
        cypher = "MATCH (src:Person {id:" + "'" + src + "'}),"
        cypher = cypher + " (tar:Person {id:" + "'" + tar + "'})"
        cypher = cypher + \
            "MERGE (src)-[r:INTERACTS_"+ str(season) +"]-(tar)"
        cypher = cypher + f"SET r.weight={weight} SET r.season={season}"
        graph.run(cypher)
    return "Done creating edges"

In [10]:
def create_graph():
    """Function to create the graph from the csv files.

    Returns:
        str: Acknowledgement string
    """
    for season in range(1,9):
        for type in ['nodes','edges']:
            base_url = "https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/"
            final_url = base_url+"got-s"+str(season)+"-"+type+".csv"
            df = pd.read_csv(final_url)
            if type == 'nodes':
                create_nodes(df, season)
            else:
                create_edges(df)
    return "Done creating graph"

In [11]:
create_graph()

'Done creating graph'